In [21]:
PATHS = (
    ("/tmp/numered_all.json", "/tmp/gpt4o-generated-speech"),
    ("/tmp/ggpt_numbered.json", "/tmp/groundinggpt-generated-speech"),
    ("/tmp/ggpt_numbered2.json", "/tmp/groundinggpt-generated-speech2")
)

In [1]:
import json

with open("/tmp/numered_all.json") as inf:
    data = json.load(inf)

In [9]:
from string import punctuation

punct = set(punctuation)

In [ ]:
def gather_phrase(text):
    words = text.split(" ")
    phrase = []
    inphrase = False
    start = 0
    for idx, word in enumerate(words):
        if inphrase:
            if word.endswith("*") or (word[-1] in punct and word[-2] == "*"):
                word = word.replace("*", "")
                phrase.append(word)
                inphrase = False
            else:
                phrase.append(word)
        else:
            if word.startswith("*"):
                if word.endswith("*") or (word[-1] in punct and word[-2] == "*"):
                    return idx, word.replace("*", "")
                else:
                    phrase.append(word.strip("*"))
                    start = idx
                inphrase = True
            else:
                continue
    return start, (" ".join(phrase)).strip("*")

In [ ]:
def index_of_determiner(phrase):
    dp = gather_phrase(phrase)
    words = phrase.split(" ")
    for i, word in enumerate(words):
        if word in ["this", "that", "these", "those"]:
            return i
    return -1

In [ ]:
def clean_text(text):
    text = text.replace("’", "'")
    text = text.replace("—", " ")
    words = text.split(" ")
    cleaned = [w.lower().strip(punctuation) for w in words]
    return " ".join(cleaned)

In [ ]:
!pip install jiwer

In [24]:
from jiwer import wer

In [22]:
def prune_fillers(text):
    FILLERS = ["uh", "um"]
    words = [x for x in text.split(" ") if x not in FILLERS]
    return " ".join(words)

In [ ]:
FIXES = {
    "hsi_4_0717_222_002__0__8": ("racket", "racquet")
}

In [49]:
def read_tsv(tsvfile):
    tsvdata = []
    with open(tsvfile) as inf:
        for line in inf:
            line = line.strip()
            if not line:
                continue
            tsvdata.append(line.split("\t"))
    return tsvdata

In [50]:
from pathlib import Path

THRESHOLD = 0.3

def process_data(data, tsvfile, gatherable_phrase = True):
    for item in data:
        person = item["person"]
        fileid = item["id"]
        text = item["snippet"]

        # tsvfile = Path("/tmp") / f"gpt4o-generated-speech" / f"{fileid}.tsv"
        tsvdata = read_tsv(tsvfile)
        
        tsvwords = [x[2] for x in tsvdata]
        tsvtext = " ".join(tsvwords)

        cleaned_text = clean_text(text.strip())
        cleaned_tsv = clean_text(tsvtext)

        if gather_phrase:
            phrase = gather_phrase(text)
            phrase_parts = phrase.split(" ")
            if phrase_parts[0] in ["a", "the"]:
                discard_ids.append(fileid)
                continue

        if cleaned_text != cleaned_tsv:
            if cleaned_tsv.replace("-", " ") == cleaned_text:
                cleaned_tsv = cleaned_tsv.replace("-", " ")

        discard_ids = set()

        if cleaned_text != cleaned_tsv:
            if gather_phrase and not phrase in cleaned_tsv:
                print("Phrase not in tsv", phrase)
                discard_ids.append(fileid)
            print("Mismatch", fileid, person)
            print(cleaned_text)
            print(cleaned_tsv)
            cur_wer = wer(cleaned_text, cleaned_tsv)
            print(cur_wer)
            if cur_wer > THRESHOLD:
                discard_ids.append(fileid)
                print("WER above threshold, discarding")
            print()

        # print(gather_phrase(text), "\t", text)


In [47]:
discard_ids

['hsi_5_0718_211_003__50__41',
 'hsi_5_0718_211_003__50__42',
 'hsi_5_0718_211_003__50__43']